In [2]:
import os 
import allel
import numpy as np 
import pandas as pd

In [3]:
cd ../

/Users/anushasubramanian


In [4]:
cd Desktop/Research/UCLA/tsv

/Users/anushasubramanian/Desktop/Research/UCLA/tsv


In [5]:
len(os.listdir())

7704

In [6]:
f = pd.read_csv('/Users/anushasubramanian/Desktop/Research/UCLA/yeast_parent_vcf/sgs1_as_csv.csv')
sgs1 = pd.DataFrame(f)
keys = ["chrI","chrII","chrIII","chrIV","chrV","chrVI","chrVII","chrVIII","chrIX","chrX",
        "chrXI","chrXII","chrXIII","chrXIV", "chrXV","chrXVI","chrM"]
sgs1PosDict = dict.fromkeys(keys)

In [7]:
for k in sgs1PosDict:
            sgs1PosDict[k] = list(sgs1.loc[sgs1["CHROM"] == k]["POS"])

In [8]:
sgs1PosDict["chrIV"][-1]

1525226

In [9]:
chrIVPos = dict.fromkeys(list(sgs1PosDict["chrIV"]), 0)

In [10]:
chrIVCoverage = dict.fromkeys(list(sgs1PosDict["chrIV"]), 0)

In [18]:
## Function to generate a full dataset from each file 

def full_data(file):
    
    """ To generate the full data associated with a vcf file passed in """
    
    data = pd.read_csv( file, sep = '\t')
    chrIV = data.loc[data["#CHROM"] == "chrIV"] # all chrIV only
    chrIV.columns = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT','psl5',
                   'psl2','yi283']


    # only use positions from sgs1
    for pos in chrIV["POS"]:
        if pos not in sgs1PosDict["chrIV"]:
            chrIV.drop(chrIV.loc[chrIV["POS"] == pos].index, inplace = True)

    #3. Cleaning the yi283 data to be in usable form, with ref and alt reads separated
    ref = []
    alt = []
    all_ref = []
    all_alt = []

    error = []
    for item in chrIV["yi283"]:
        reads = item.split(sep = ':')[1:3]
        if len(reads) == 2:
            for read in reads: 
                tokens = read.split(sep = ',')
                ref.append(tokens[0])
                alt.append(tokens[1])
            all_ref.append([int(x) for x in ref])
            all_alt.append([int(x) for x in alt])
            ref = []
            alt = []   
        else:
            error.append(item)
            
    # Finding sum of the reference and alternative reads 
    sum_ref = []
    sum_alt = []

    for ref in all_ref:
        sum_ref.append(sum(ref))
    for alt in all_alt:
        sum_alt.append(sum(alt))


    # finding the total number of reads by summing all ref and alts
    zipped = zip(sum_ref, sum_alt)
    tot_reads = [x+y for (x,y) in zipped]

    cleaned_chrIV = pd.DataFrame()
    cleaned_chrIV["CHROM"] = chrIV["CHROM"]
    cleaned_chrIV["POS"] = chrIV["POS"]
    cleaned_chrIV["total reads"] = tot_reads
    cleaned_chrIV["ref reads"] = all_ref
    cleaned_chrIV["alt reads"] = all_alt
    cleaned_chrIV["sum_ref"] = sum_ref
    cleaned_chrIV["sum_alt"] = sum_alt
    

    # percentage of ref = sum ref / total reads
    refPer = []
    altPer = []

    for ref, reads in zip(sum_ref, tot_reads):
        refPer.append((ref/reads)*100)
    for alt, reads in zip(sum_alt, tot_reads):
        altPer.append((alt/reads)*100)

    cleaned_chrIV["%ref"] = refPer
    cleaned_chrIV["%alt"] = altPer

    # quick check to make sure that all the entries are either 0 or 100
    e = False
    for alt in altPer:
        if alt!=0 and alt!=100:
            e = True
            
    #reset the index so we can access multiple columns of df
    cleaned_chrIV.reset_index(drop = True, inplace=True)
    
    # Classify each position as alt or ref
    classify = []
    for ind in cleaned_chrIV.index:
        if cleaned_chrIV["%ref"][ind] > cleaned_chrIV["%alt"][ind]:
            classify.append("ref")
        elif cleaned_chrIV["%alt"][ind] > cleaned_chrIV["%ref"][ind]:
            classify.append("alt")
        else:
            classify.append('ref/alt')
            
    cleaned_chrIV["pos classification"] = classify
    
    chrIV.drop(columns=["ID","REF","ALT","QUAL","FILTER"], inplace=True)
    
    
    # Get whether positions are alt or ref for PSL5
    
    ref5 = []
    alt5 = []
    all_ref5 = []
    all_alt5 = []

    error = []
    for item in chrIV["psl5"]:
        reads = item.split(sep = ':')[1:3]
        if len(reads) == 2:
            for read in reads:
                tokens = read.split(sep = ',')
                ref5.append(tokens[0])
                alt5.append(tokens[1])
            all_ref5.append([int(x) for x in ref5])
            all_alt5.append([int(x) for x in alt5])
            ref5 = []
            alt5 = []   
        else:
            error.append(item)
            
    #Finding sum of the reference and alternative reads in psl5
    sum_ref5 = []
    sum_alt5 = []

    for ref in all_ref5:
        sum_ref5.append(sum(ref))
    for alt in all_alt5:
        sum_alt5.append(sum(alt))

    # finding the total number of reads by summing all ref and alts
    zipped = zip(sum_ref5, sum_alt5)
    tot_reads5 = [x+y for (x,y) in zipped]
    
    
    
    # percentage of ref = sum ref / total reads

    refPer5 = []
    altPer5 = []

    for ref, reads in zip(sum_ref5, tot_reads5):
        refPer5.append((ref/reads)*100)
    for alt, reads in zip(sum_alt5, tot_reads5):
        altPer5.append((alt/reads)*100)



    classify5 = []
    for alt,ref in zip(altPer5, refPer5):
        if ref > alt:
            classify5.append("ref")
        elif alt > ref:
            classify5.append("alt")
        else:
            classify5.append('ref/alt')
    
    
    cleaned_chrIV["psl5 pos"] = classify5
    
    # Get whether positions are alt or ref for PSL2
    ref2 = []
    alt2 = []
    all_ref2 = []
    all_alt2 = []

    error = []
    for item in chrIV["psl2"]:
        reads = item.split(sep = ':')[1:3]
        if len(reads) == 2:
            for read in reads:
                tokens = read.split(sep = ',')
                ref2.append(tokens[0])
                alt2.append(tokens[1])
            all_ref2.append([int(x) for x in ref2])
            all_alt2.append([int(x) for x in alt2])
            ref2 = []
            alt2 = []   
        else:
            error.append(item)


    #Finding sum of the reference and alternative reads in psl5
    sum_ref2 = []
    sum_alt2 = []

    for ref in all_ref2:
        sum_ref2.append(sum(ref))
    for alt in all_alt2:
        sum_alt2.append(sum(alt))


    # finding the total number of reads by summing all ref and alts
    zipped = zip(sum_ref2, sum_alt2)
    tot_reads2 = [x+y for (x,y) in zipped]


    # percentage of ref = sum ref / total reads

    refPer2 = []
    altPer2 = []

    for ref, reads in zip(sum_ref2, tot_reads2):
        if reads!= 0:
            refPer2.append((ref/reads)*100)
        else:
            refPer2.append(999)

    for alt, reads in zip(sum_alt2, tot_reads2):
        if reads!= 0:
            altPer2.append((alt/reads)*100)
        else:
            altPer2.append(999)


    

    classify2 = []
    for alt,ref in zip(altPer2, refPer2):
        if ref > alt:
            classify2.append("ref")
        elif alt > ref:
            classify2.append("alt")
        elif alt == 999 and ref == 999:
            classify2.append('no reads')
        else:
            classify2.append('ref/alt')


    cleaned_chrIV["psl2 pos"] = classify2
    
    # So what strain is the yi283 position from?
    
    strain = []
    for ind in cleaned_chrIV.index:
        if cleaned_chrIV["pos classification"][ind] == 'ref/alt':
            strain.append('psl5/psl2')
        else:
            if cleaned_chrIV["pos classification"][ind] == cleaned_chrIV["psl5 pos"][ind]:
                strain.append("psl5")

            elif cleaned_chrIV["pos classification"][ind] == cleaned_chrIV["psl2 pos"][ind]:
                strain.append("psl2")
            else:
                strain.append("inconclusive")
                
    cleaned_chrIV["strain"] = strain
    
    psl5Per = []
    psl2Per = []

    for ind in cleaned_chrIV.index:

        if cleaned_chrIV["strain"][ind] == 'psl5/psl2':
            psl5Per.append(cleaned_chrIV["%ref"][ind])
            psl2Per.append(cleaned_chrIV["%alt"][ind])

        elif cleaned_chrIV["strain"][ind] == "psl5":

            if cleaned_chrIV["pos classification"][ind] == 'alt':
                psl5Per.append(cleaned_chrIV["%alt"][ind])
                psl2Per.append(cleaned_chrIV["%ref"][ind])

            elif cleaned_chrIV["pos classification"][ind] == 'ref':
                psl5Per.append(cleaned_chrIV["%ref"][ind])
                psl2Per.append(cleaned_chrIV["%alt"][ind])

        elif cleaned_chrIV["strain"][ind] == "psl2":

            if cleaned_chrIV["pos classification"][ind] == 'alt':
                psl2Per.append(cleaned_chrIV["%alt"][ind])
                psl5Per.append(cleaned_chrIV["%ref"][ind])

            elif cleaned_chrIV["pos classification"][ind] == 'ref':
                psl2Per.append(cleaned_chrIV["%ref"][ind])
                psl5Per.append(cleaned_chrIV["%alt"][ind])

        elif cleaned_chrIV["strain"][ind] == 'inconclusive':
            psl5Per.append(-1)
            psl2Per.append(-1)
            
    cleaned_chrIV["psl5%"] = psl5Per
    cleaned_chrIV["psl2%"] = psl2Per
    
    return cleaned_chrIV
    

In [26]:
tsv_names = [x for x in os.listdir() if x.find(".tsv") != -1]

# single_cell_data = dict.fromkeys(tsv_names, 0)

In [37]:
for tsv in tsv_names:
    df = full_data(tsv)
    df.to_csv(f'/Users/anushasubramanian/Desktop/Research/UCLA/single cell aggregated data/{tsv}', 
              sep = '\t', index = False)
    